In [1]:
import pandas as pd
import numpy as np
import regex as re
import pickle
from collections import defaultdict

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import GridSearchCV
import pyLDAvis.sklearn
import pyLDAvis
pyLDAvis.enable_notebook()

import gensim
from gensim.summarization import keywords, summarize, mz_keywords
from gensim.models import doc2vec


/Users/kristigourlay/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
data = pd.read_csv('data/legal_dataframe.csv', index_col=0)

/Users/kristigourlay/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.shape

(183149, 13)

In [4]:
data.head()

,case_id,name,name_abbreviation,decision_date,first_page,last_page,case_citation_name,volume,reporter,court,jurisdiction,casebody,text
0,2747110,"The People of the State of Illinois, Plaintiff...",People v. Tobin,1771-10-12,538,543,2 Ill. App. 3d 538,2,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Mr. PRESIDING JUSTICE EBERSPACHER\ndelivered t...
1,435537,"James A. Whitesides and others, Plaintiffs in ...",Whitesides v. People,1819-12,21,22,1 Ill. 21,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court. This was a criminal pros...
2,435638,"Amos Chipps, Appellant, v. Thomas Yancey, Appe...",Chipps v. Yancey,1819-12,19,19,1 Ill. 19,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n*\nThis was an action o...
3,435690,"Jonathan Taylor, Appellant, v. Michael Sprinkl...",Taylor v. Sprinkle,1819-12,17,18,1 Ill. 17,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n*\nThis was an action o...
4,435710,"François Coleen and Abraham Claypole, Appellan...",Coleen v. Figgins,1819-12,19,20,1 Ill. 19,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n†\nIt appears from the ...


In [5]:
data.tail()

,case_id,name,name_abbreviation,decision_date,first_page,last_page,case_citation_name,volume,reporter,court,jurisdiction,casebody,text
183144,4289128,"GENERAL MOTORS CORPORATION et al., Appellees, ...",General Motors Corp. v. Pappas,2011-05-19,163,189,242 Ill. 2d 163,242,"Illinois Reports, Second Series",Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",JUSTICE THEIS\ndelivered the judgment of the c...
183145,7328627,"HARRY BALOUGH, Plaintiff-Appellee, v. NORTHEAS...",Balough v. Northeast Illinois Regional Commute...,2011-05-19,750,780,409 Ill. App. 3d 750,409,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",JUSTICE PUCINSKI\ndelivered the judgment of th...
183146,7328860,"In re CHARLES H., a Person Found Subject to In...",People v. Charles H.,2011-05-20,1047,1058,409 Ill. App. 3d 1047,409,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",JUSTICE POPE\ndelivered the judgment of the co...
183147,7328871,"THE PEOPLE OF THE STATE OF ILLINOIS, Plaintiff...",People v. Isaacson,2011-05-20,1079,1086,409 Ill. App. 3d 1079,409,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",JUSTICE TURNER\ndelivered the judgment of the ...
183148,7328886,"In re ESTATE OF RONALD D. WEEKS, Deceased (Dav...",Hammer v. People ex rel. Madigan,2011-05-20,1101,1114,409 Ill. App. 3d 1101,409,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",PRESIDING JUSTICE KNECHT\ndelivered the judgme...


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183149 entries, 0 to 183148
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   case_id             183149 non-null  int64 
 1   name                183149 non-null  object
 2   name_abbreviation   183149 non-null  object
 3   decision_date       183149 non-null  object
 4   first_page          183149 non-null  object
 5   last_page           183149 non-null  object
 6   case_citation_name  183149 non-null  object
 7   volume              183149 non-null  int64 
 8   reporter            183149 non-null  object
 9   court               183149 non-null  object
 10  jurisdiction        183149 non-null  object
 11  casebody            183149 non-null  object
 12  text                183149 non-null  object
dtypes: int64(2), object(11)
memory usage: 19.6+ MB


In [7]:
data.describe()

,case_id,volume
count,1.831490e+05,183149.000000
mean,3.710537e+06,159.692010
std,1.617843e+06,114.108253
min,2.533200e+04,1.000000
25%,2.700087e+06,54.000000
50%,3.305630e+06,145.000000
75%,5.229687e+06,250.000000
max,1.225594e+07,415.000000


# Extracting Citations

In [14]:
article_citations = []

for item in data.text:
    
    item_cite = []
    
    try:
        item_cite.extend(re.findall(r'\(\d{4}\),\s\d+\s\w+.\s\dd\s\d+', item))
      
    except ExplicitException:
        pass
    
    try:
        item_cite.extend(re.findall(r'\(\d{4}\),\s\d+\s\w+.\s\w+.\s\w+\s\d+', item))

    except ExplicitException:
        pass
    
    try:
        item_cite.extend(re.findall(r'\(\d{4}\),\s\d+\s\w+.\s\w+.\s\d{2,3}', item))

    except ExplicitException:
        pass
    
    try:
        item_cite.extend(re.findall(r'\d+\s\D{2,4}.\s\d{2,5}', item))
        
    except: 
        pass
    
    article_citations.append(list(set(item_cite)))
    if len(article_citations) % 10000 == 0:
        print(len(article_citations))
        
data['citations'] = article_citations

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000


In [21]:
data.citations = data.citations.apply(lambda y: np.nan if (type(y) == list and len(y) == 0) else y)

# Clean Text Data

In [27]:
class Datacleaner(TransformerMixin):
    

    def fit(self, raw_text):
        return self
    
    def clean(self, raw_text):
        
        tokenizer = RegexpTokenizer(r'\w+')
        lemmatizer = WordNetLemmatizer()
        
        clean_text = BeautifulSoup(raw_text).get_text()
        lower_cases = clean_text.lower()
        tokens = tokenizer.tokenize(lower_cases)
        lemms = [lemmatizer.lemmatize(word) for word in tokens]
        words = [word for word in lemms if word not in stopwords.words('english')]
        final_words = ' '.join(words)
        
        return final_words
    
    def clean_col(self, col):
        
        clean_list = []
        n = 0
        for item in col:
            n += 1
            if n % 10000 == 0:
                print(n)
            clean_list.append( self.clean(item) )
            
        return clean_list
        

In [32]:
cleaner = Datacleaner()

data['clean_text'] = cleaner.clean_col(data.text)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000


In [36]:
data.clean_text[0][:100]

'mr presiding justice eberspacher delivered opinion court defendant tobin wa convicted jury crime bur'

In [2]:
# data = pd.read_csv('data/backup.csv', index_col=0)

/Users/kristigourlay/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Bag of Words (Also finding corpus specfic stopwords)

In [41]:
cvec = CountVectorizer(max_features=50, strip_accents='unicode')
non_dense_count_df = cvec.fit_transform(data.clean_text)
count_df = pd.DataFrame(cvec.fit_transform(data.clean_text).todense(), columns=cvec.get_feature_names())

In [42]:
most_common_words = dict(count_df.sum().sort_values()[::-1])

In [43]:
stop_words = list(stopwords.words('english'))

sw = list(most_common_words.keys())[:29]

In [44]:
stop_words.extend(sw)

# Summarization & Keywords

In [55]:
data.head()

,case_id,name,name_abbreviation,decision_date,first_page,last_page,case_citation_name,volume,reporter,court,jurisdiction,casebody,text,citations,clean_text,keywords
0,2747110,"The People of the State of Illinois, Plaintiff...",People v. Tobin,1771-10-12,538,543,2 Ill. App. 3d 538,2,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Mr. PRESIDING JUSTICE EBERSPACHER\ndelivered t...,"[1 and 119, 89 S.Ct. 2100, 395 U.S. 959]",mr presiding justice eberspacher delivered opi...,"[sentenced, sentence, sentencing, court defend..."
1,435537,"James A. Whitesides and others, Plaintiffs in ...",Whitesides v. People,1819-12,21,22,1 Ill. 21,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court. This was a criminal pros...,NaN,opinion court wa criminal prosecution riot pla...,"[indictment, form, criminal, shall, year]"
2,435638,"Amos Chipps, Appellant, v. Thomas Yancey, Appe...",Chipps v. Yancey,1819-12,19,19,1 Ill. 19,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n*\nThis was an action o...,"[2 Dall, 302]",opinion court wa action debt judgment rendered...,"[action, court, judgment, plea, nil]"
3,435690,"Jonathan Taylor, Appellant, v. Michael Sprinkl...",Taylor v. Sprinkle,1819-12,17,18,1 Ill. 17,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n*\nThis was an action o...,NaN,opinion court wa action covenant fifth plea st...,"[plea, court, post, failure, illinois]"
4,435710,"François Coleen and Abraham Claypole, Appellan...",Coleen v. Figgins,1819-12,19,20,1 Ill. 19,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n†\nIt appears from the ...,NaN,opinion court appears record cause writ issued...,"[court appears, scam, appearance, appearing, p..."


In [52]:
keywords(data['clean_text'][0], split=True, ratio=.03)

['sentenced',
 'sentence',
 'sentencing',
 'court defendant tobin',
 'building',
 'ill',
 'state',
 'evidence',
 'stated car',
 'authority',
 'door']

In [53]:
kw = []
n = -1
for item in data.clean_text:
    n += 1
    if n % 10000 == 0:
        print(f'Fetching keywords for #{n}')
    if len(item) > 100:
        try:
            kw.append(keywords(item, split=True, words=5))
        except:
            kw.append(keywords(item, split=True, ratio=.1))

    else:
        kw.append(item)


Fetching keywords for #0
Fetching keywords for #10000
Fetching keywords for #20000
Fetching keywords for #30000
Fetching keywords for #40000
Fetching keywords for #50000
Fetching keywords for #60000
Fetching keywords for #70000
Fetching keywords for #80000
Fetching keywords for #90000
Fetching keywords for #100000
Fetching keywords for #110000
Fetching keywords for #120000
Fetching keywords for #130000
Fetching keywords for #140000
Fetching keywords for #150000
Fetching keywords for #160000
Fetching keywords for #170000
Fetching keywords for #180000


In [54]:
data['keywords'] = kw

In [57]:
summarize(data.text[0], word_count=250).replace('\n', '')

'The court entered judgment upon the verdict and sentenced the defendant to a fifteen to twenty-five year term in the Illinois State Penitentiary.The defendant has appealed from that judgment and raised the following issues: (1) The State failed to prove lack of authority to enter the premises; (2) The State failed to prove intent to commit a theft; (3) The court erred in allowing testimony concerning the arrest of Sherri Tobin, her possession of a firearm and evidence concerning defendant’s possession of a firearm; (4) The sentence was excessive.The facts giving rise to this case are as follows: On the night of February 9, 1969, at about 11:00 P.M. the defendant, in the company of Sherri Tobin, Daniel Stout, Michael Hume and Eddie Dunn was in an automobile driven by defendant in the vicinity of the Oliver C.He stated the car stopped by the agency and the driver, identified as Tobin, jumped out and ran across the street and kicked the agency door.In regard to the question of intent the

In [7]:
summs = []
n = 0

for item in data.text:
    n+= 1
    if n % 10000 == 0:
        print(f'Fetching summary for #{n}')
        
    if len(item) > 1000:
        try:
            summs.append(summarize(item, word_count=250).replace('\n', ''))
        except:
            summs.append(summarize(item, ratio=.5).replace('\n', ''))
    else:
        summs.append(item)


Fetching summary for #10000
Fetching summary for #20000
Fetching summary for #30000
Fetching summary for #40000
Fetching summary for #50000
Fetching summary for #60000
Fetching summary for #70000
Fetching summary for #80000
Fetching summary for #90000
Fetching summary for #100000
Fetching summary for #110000
Fetching summary for #120000
Fetching summary for #130000
Fetching summary for #140000
Fetching summary for #150000
Fetching summary for #160000
Fetching summary for #170000
Fetching summary for #180000


In [8]:
data['summs'] = summs

In [13]:
data.summs.fillna(data.text, inplace=True)

# Doc2Vec

In [14]:
def tag_docs(documents):
    
    for i, line in enumerate(documents):
        tokens = gensim.utils.simple_preprocess(line)
        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [15]:
train_data = list(tag_docs(data.clean_text))

In [16]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [17]:
model.build_vocab(train_data)

In [18]:
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [19]:
model

In [20]:
example = train_data[0]

In [21]:
sample_vector = model.infer_vector(example.words)
sample_vector

array([ 1.5324306 ,  2.1731114 ,  0.5358954 ,  0.69633293,  0.59287786,
       -0.56913793, -0.8112813 , -0.91343445,  0.36389783,  1.1947542 ,
        1.4784077 ,  1.4262766 ,  0.78247553,  0.53290284, -0.0711604 ,
       -0.9184037 ,  0.47452053,  0.6324937 , -1.0646518 ,  0.2243761 ,
        0.04560428, -0.43814626, -0.4730711 ,  1.1807055 , -1.2522606 ,
        1.5006284 ,  1.4920942 , -1.4339331 , -1.7539495 ,  1.9458628 ,
       -0.58788145, -0.3910977 ,  0.274218  ,  0.8812365 , -2.36569   ,
        1.3564054 , -1.8362895 , -0.05379141, -0.6475554 , -0.37915578,
        0.5082969 ,  0.65028775,  0.5514563 , -1.1354003 , -0.6905312 ,
       -0.07784279, -1.0563327 , -0.5240961 ,  1.1317965 , -0.56958365],
      dtype=float32)

In [22]:
n = 0
for item in model.docvecs.most_similar([sample_vector])[1:]:
    n += 1

    print(f'Most Similar Case # {n}: \nCase #{item[0]}\nSimilarity Score: {item[1]}\nKeywords: {data.keywords[item[0]]}\n\nCase Summary:\n{data.summs[item[0]]}\n\n')

Most Similar Case # 1: 
Case #153648
Similarity Score: 0.7980687618255615
Keywords: ['defendant', 'sentenced', 'sentence', 'sentencing', 'court', 'said', 'trial']

Case Summary:
On appeal, defendant maintains: (1) he was not proved guilty beyond a reasonable doubt; (2) there was no probable cause to arrest defendant, and, therefore, the court erred in denying defendant’s motion to suppress items seized without a warrant and statements made subsequent to the arrest; (3) defendant was deprived of his right to a fair trial by comments made by the State during closing argument; and (4) the trial court erred in sentencing defendant to consecutive terms of imprisonment.He said he then conducted a search of the area and found a green Army duffel bag, an aluminum suitcase, a green toolbox, and a six pack of beer in a railroad “subway ditch,” which was approximately 100 yards from the place where defendant was being detained.He said he then, at 5:15 a.m., instructed Officer Beason to place defe

In [23]:
polarity_scores = []

for i in range(0, len(data)):
    vector = model.infer_vector(train_data[i].words)[0]
    polarity_scores.append(vector)

In [24]:
data['d2v_scores'] = polarity_scores

In [25]:
data.head(1)

,case_id,name,name_abbreviation,decision_date,first_page,last_page,case_citation_name,volume,reporter,court,jurisdiction,casebody,text,citations,clean_text,keywords,summs,d2v_scores,lda_preds
0,2747110,"The People of the State of Illinois, Plaintiff...",People v. Tobin,1771-10-12,538,543,2 Ill. App. 3d 538,2,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Mr. PRESIDING JUSTICE EBERSPACHER\ndelivered t...,"['1 and 119', '89 S.Ct. 2100', '395 U.S. 959']",mr presiding justice eberspacher delivered opi...,"['sentenced', 'sentence', 'sentencing', 'court...",The court entered judgment upon the verdict an...,1.3064,4


# LDA Topic Modeling

In [26]:
data.head()

,case_id,name,name_abbreviation,decision_date,first_page,last_page,case_citation_name,volume,reporter,court,jurisdiction,casebody,text,citations,clean_text,keywords,summs,d2v_scores,lda_preds
0,2747110,"The People of the State of Illinois, Plaintiff...",People v. Tobin,1771-10-12,538,543,2 Ill. App. 3d 538,2,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Mr. PRESIDING JUSTICE EBERSPACHER\ndelivered t...,"['1 and 119', '89 S.Ct. 2100', '395 U.S. 959']",mr presiding justice eberspacher delivered opi...,"['sentenced', 'sentence', 'sentencing', 'court...",The court entered judgment upon the verdict an...,1.306400,4
1,435537,"James A. Whitesides and others, Plaintiffs in ...",Whitesides v. People,1819-12,21,22,1 Ill. 21,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court. This was a criminal pros...,NaN,opinion court wa criminal prosecution riot pla...,"['indictment', 'form', 'criminal', 'shall', 'y...","This was a criminal prosecution for a riot, ag...",1.451599,4
2,435638,"Amos Chipps, Appellant, v. Thomas Yancey, Appe...",Chipps v. Yancey,1819-12,19,19,1 Ill. 19,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n*\nThis was an action o...,"['2 Dall, 302']",opinion court wa action debt judgment rendered...,"['action', 'court', 'judgment', 'plea', 'nil']",Opinion of the Court.This was an action of deb...,0.455963,3
3,435690,"Jonathan Taylor, Appellant, v. Michael Sprinkl...",Taylor v. Sprinkle,1819-12,17,18,1 Ill. 17,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n*\nThis was an action o...,NaN,opinion court wa action covenant fifth plea st...,"['plea', 'court', 'post', 'failure', 'illinois']",Opinion of the Court.This was an action of cov...,0.994830,3
4,435710,"François Coleen and Abraham Claypole, Appellan...",Coleen v. Figgins,1819-12,19,20,1 Ill. 19,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n†\nIt appears from the ...,NaN,opinion court appears record cause writ issued...,"['court appears', 'scam', 'appearance', 'appea...","It appears from the record in this cause, that...",0.522237,3


In [45]:
cvec = CountVectorizer(strip_accents='unicode',
                       min_df=5,
                        stop_words=stop_words) #stop_words include legal corpus specific 

In [46]:
non_dense_count_df = cvec.fit_transform(data.clean_text)

In [31]:
# count_df = pd.DataFrame(non_dense_count_df.todense(), columns=cvec.get_feature_names())
# count_df.head()

In [32]:
lda_model = LDA(n_components=5, random_state=42)

In [33]:
lda_model.fit(non_dense_count_df)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [34]:
pyLDAvis.sklearn.prepare(lda_model, non_dense_count_df, cvec)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.111801  0.075143       1        1  26.247609
0     -0.031903 -0.139163       2        1  22.010270
3      0.148891 -0.053922       3        1  21.147886
1     -0.087863 -0.013129       4        1  16.413068
4      0.082676  0.131071       5        1  14.181167, topic_info=            Term           Freq          Total Category  logprob  loglift
47067       jury  434280.000000  434280.000000  Default  30.0000  30.0000
22081   contract  340847.000000  340847.000000  Default  29.0000  29.0000
9557   appellant  393535.000000  393535.000000  Default  28.0000  28.0000
44713     injury  216704.000000  216704.000000  Default  27.0000  27.0000
66182     policy  176137.000000  176137.000000  Default  26.0000  26.0000
...          ...            ...            ...      ...      ...      ...
62077    opinion   68733.629425  356801.462979   Topic5  -5.7513   0.3063
83426  testimony   66815.335539  336091.960463   Topic5  -5.7796   0.3378
91654    witness   63589.806786  299864.870728   Topic5  -5.8291   0.4024
31408      error   63573.410768  347834.554872   Topic5  -5.8293   0.2537
90814    whether   63593.928620  397791.817596   Topic5  -5.8290   0.1198

[534 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
1          1  0.497905     000
1          2  0.065861     000
1          3  0.008005     000
1          4  0.365085     000
1          5  0.063142     000
...      ...       ...     ...
92559      5  0.082725    year
93211      1  0.000332  zoning
93211      2  0.001361  zoning
93211      4  0.997208  zoning
93211      5  0.001095  zoning

[1517 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2, 5])

In [35]:
col_names = ['topic1', 'topic2', 'topic3', 'topic4', 'topic5']
lda_preds = pd.DataFrame(lda_model.transform(non_dense_count_df), columns=col_names)
lda_preds.head()

,topic1,topic2,topic3,topic4,topic5
0,0.136311,0.041874,0.039036,0.782521,0.000259
1,0.001306,0.001298,0.484313,0.511779,0.001304
2,0.002259,0.002252,0.991028,0.002232,0.002229
3,0.175706,0.001527,0.819710,0.001532,0.001525
4,0.148262,0.000771,0.849429,0.000769,0.000769


In [36]:
lda_topic_preds = []

for i in range(0, len(lda_preds)):
    sims = sorted(enumerate(lda_preds.iloc[i], 1), key=lambda x: x[1], reverse=True)
    lda_topic_preds.append(sims[0][0])
    
data['lda_preds'] = lda_topic_preds

In [37]:
def ldamodel(query):
    '''
    input: user query
    output: lda topic it belongs to
    '''
    count_query = cvec.transform(query)
    topic_likelihood = lda_model.transform(count_query)[0]
    topic = sorted(enumerate(topic_likelihood, 1), key=lambda x: x[1], reverse=True)[0][0]
    
    return topic

In [38]:
ldamodel(['realestate property mortgage fine'])

3

In [39]:
data.head()

,case_id,name,name_abbreviation,decision_date,first_page,last_page,case_citation_name,volume,reporter,court,jurisdiction,casebody,text,citations,clean_text,keywords,summs,d2v_scores,lda_preds
0,2747110,"The People of the State of Illinois, Plaintiff...",People v. Tobin,1771-10-12,538,543,2 Ill. App. 3d 538,2,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Mr. PRESIDING JUSTICE EBERSPACHER\ndelivered t...,"['1 and 119', '89 S.Ct. 2100', '395 U.S. 959']",mr presiding justice eberspacher delivered opi...,"['sentenced', 'sentence', 'sentencing', 'court...",The court entered judgment upon the verdict an...,0.283677,4
1,435537,"James A. Whitesides and others, Plaintiffs in ...",Whitesides v. People,1819-12,21,22,1 Ill. 21,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court. This was a criminal pros...,NaN,opinion court wa criminal prosecution riot pla...,"['indictment', 'form', 'criminal', 'shall', 'y...","This was a criminal prosecution for a riot, ag...",-0.579995,4
2,435638,"Amos Chipps, Appellant, v. Thomas Yancey, Appe...",Chipps v. Yancey,1819-12,19,19,1 Ill. 19,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n*\nThis was an action o...,"['2 Dall, 302']",opinion court wa action debt judgment rendered...,"['action', 'court', 'judgment', 'plea', 'nil']",Opinion of the Court.This was an action of deb...,-0.733031,3
3,435690,"Jonathan Taylor, Appellant, v. Michael Sprinkl...",Taylor v. Sprinkle,1819-12,17,18,1 Ill. 17,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n*\nThis was an action o...,NaN,opinion court wa action covenant fifth plea st...,"['plea', 'court', 'post', 'failure', 'illinois']",Opinion of the Court.This was an action of cov...,-0.508408,3
4,435710,"François Coleen and Abraham Claypole, Appellan...",Coleen v. Figgins,1819-12,19,20,1 Ill. 19,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court.\n†\nIt appears from the ...,NaN,opinion court appears record cause writ issued...,"['court appears', 'scam', 'appearance', 'appea...","It appears from the record in this cause, that...",-0.371394,3


In [38]:
# data.to_csv('backup.csv')

In [47]:
# pickle.dump(lda_model, open('models/lda_final_model', 'wb'))
# pickle.dump(cvec, open('models/final_cvec_model', 'wb'))
# pickle.dump(model, open('models/d2v_final_model', 'wb'))

# Final Revisions to Dataframe

In [29]:
data.head(2)

,case_id,name,name_abbreviation,decision_date,first_page,last_page,case_citation_name,volume,reporter,court,jurisdiction,casebody,text,citations,clean_text,keywords,summs,d2v_scores,lda_preds
0,2747110,"The People of the State of Illinois, Plaintiff...",People v. Tobin,1771-10-12,538,543,2 Ill. App. 3d 538,2,"Illinois Appellate Court Reports, Third Series",Illinois Appellate Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Mr. PRESIDING JUSTICE EBERSPACHER\ndelivered t...,"['1 and 119', '89 S.Ct. 2100', '395 U.S. 959']",mr presiding justice eberspacher delivered opi...,"['sentenced', 'sentence', 'sentencing', 'court...",The court entered judgment upon the verdict an...,1.306400,4
1,435537,"James A. Whitesides and others, Plaintiffs in ...",Whitesides v. People,1819-12,21,22,1 Ill. 21,1,Illinois Reports,Illinois Supreme Court,Illinois,"{'status': 'ok', 'data': {'opinions': [{'type'...",Opinion of the Court. This was a criminal pros...,NaN,opinion court wa criminal prosecution riot pla...,"['indictment', 'form', 'criminal', 'shall', 'y...","This was a criminal prosecution for a riot, ag...",1.451599,4


In [30]:
data.decision_date[0]

'1771-10-12'

In [31]:
new_date = []

for item in data.decision_date:
    new_date.append(re.findall(r'\d{4}', item)[0])

In [32]:
data['decision_date'] = new_date

In [33]:
data['decision_date'] = data.decision_date.astype(int)

In [34]:
df = data[['case_id', 'decision_date', 'case_citation_name', 'court', 'citations', 'clean_text', 'keywords', 'summs', 'd2v_scores', 'lda_preds']]

In [35]:
print(df.shape)
print(data.shape)

(183149, 10)
(183149, 19)


In [36]:
df.head(2)

,case_id,decision_date,case_citation_name,court,citations,clean_text,keywords,summs,d2v_scores,lda_preds
0,2747110,1771,2 Ill. App. 3d 538,Illinois Appellate Court,"['1 and 119', '89 S.Ct. 2100', '395 U.S. 959']",mr presiding justice eberspacher delivered opi...,"['sentenced', 'sentence', 'sentencing', 'court...",The court entered judgment upon the verdict an...,1.306400,4
1,435537,1819,1 Ill. 21,Illinois Supreme Court,NaN,opinion court wa criminal prosecution riot pla...,"['indictment', 'form', 'criminal', 'shall', 'y...","This was a criminal prosecution for a riot, ag...",1.451599,4


In [37]:
# df.to_csv('data/final_legal_df.csv')